# Customer Churn Prediction
---
In this notebook, we will build and evaluate a machine learning model to predict customer churn. We'll use a Random Forest classifier and explore hyperparameter tuning to optimize its performance.

## 1. Import Libraries

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

## 2. Load Data

In [2]:
# Load your dataset (replace 'your_dataset.csv' with your actual file)
df = pd.read_csv('../data/processed/processed_customer_data.csv')

## 3. Data Preprocessing

In [3]:
# Data Preprocessing
# Assuming 'CustomerID', 'SubscriptionStart', 'SubscriptionEnd', 'LastPurchaseDate', and 'AgeGroup' are not needed for modeling
X = df.drop(['CustomerID', 'Churn', 'SubscriptionStart', 'SubscriptionEnd', 'LastPurchaseDate', 'Age'], axis=1)
y = df['Churn']  # Target variable

In [4]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Specify the column to treat as categorical
categorical_cols = ['LoginFrequency']

# One-Hot Encoding for the specified categorical variable
X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Align the training and testing data
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


## 4. Model Building & Evaluation

In [6]:
# Model Building - Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = rf_model.predict(X_test_scaled)

# Model Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

          No       1.00      1.00      1.00        13
         Yes       1.00      1.00      1.00         5

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18


Confusion Matrix:
 [[13  0]
 [ 0  5]]


## 5. Feature Importance

In [8]:
# Check the correct feature set used in the model
feature_names = X_train_encoded.columns  # Use the encoded feature names

# Feature Importance
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': rf_model.feature_importances_
})

# Ensure lengths match
if len(feature_importance) == len(feature_names):
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    print("\nTop 10 Feature Importance:\n", feature_importance.head(10))
else:
    print("Length mismatch between feature names and feature importances.")


Top 10 Feature Importance:
                    feature  importance
0           TotalPurchases    0.208981
4    DaysSinceLastPurchase    0.166916
6     AvgPurchasesPerMonth    0.156792
3                   Tenure    0.093699
94    LoginFrequency_Daily    0.062752
5    LoginFrequencyNumeric    0.058233
1            AvgOrderValue    0.054922
95  LoginFrequency_Monthly    0.052598
7             LoyaltyScore    0.051480
96   LoginFrequency_Weekly    0.035767


## 6. Hyperparameter Tuning

In [9]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

print("\nBest Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.2s
[CV] END max_depth=5, min_samples_leaf

## 7. Evaluate Best Model

In [10]:
# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)
print("\nClassification Report for Best Model:\n", classification_report(y_test, y_pred_best))


Classification Report for Best Model:
               precision    recall  f1-score   support

          No       1.00      1.00      1.00        13
         Yes       1.00      1.00      1.00         5

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18

